In [1]:
import pickle
import pandas as pd
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch
with torch.profiler.profile() as profiler:
        pass


In [2]:
import wandb

wandb.login()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)


True

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")
    

Running on the GPU


In [4]:
REBUILD_DATA = True


class GRV:
    # class to store training data

    # reading data from pickle
    file = open(r"DATA/total_list_w_price_bf.npy", "rb")
    data = pickle.load(file)
    # seperate out classes from inputs
    raceIDs, inputs, classes, prices, win_price, margins, betfairSP = zip(*data)
    # removing nan from inputs and convert to float
    inputs_df = pd.DataFrame(inputs)
    inputs_df.fillna(value=-1, inplace=True)
    inputs = inputs_df.values.tolist()
    inputs = [[float(i) for i in j] for j in inputs]

    # data
    training_data = []

    def make_training_data(self):
        excluded = 0
        for i in range(len(self.inputs)):
            if len(self.classes[i]) == 8:
                self.training_data.append(
                    [
                        np.array(self.inputs[i]),
                        np.array(self.classes[i]),
                        np.array(self.prices[i]),
                        np.array(self.margins[i]),
                        np.array(self.betfairSP[i]),
                    ]
                )
            else:
                adjustedList = self.classes[i] + ([8] * (8 - len(self.classes[i])))
                adjustedListP = self.prices[i] + ([0] * (8 - len(self.prices[i])))
                adjustedListM = self.margins[i] + ([100] * (8 - len(self.margins[i])))
                adjustedListSP = self.margins[i] + ([0] * (8 - len(self.betfairSP[i])))
                self.training_data.append(
                    [
                        np.array(self.inputs[i]),
                        np.array(adjustedList),
                        np.array(adjustedListP),
                        np.array(adjustedListM),
                        np.array(adjustedListSP)
                    ]
                )
                if len(adjustedList) != 8:
                    print(adjustedList)
        np.save("training_data.npy", self.training_data)
        print("excluded = ", excluded)


if REBUILD_DATA:
    grv = GRV()
    grv.make_training_data()


c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


excluded =  0


In [5]:
softmin = nn.Softmin(dim=1)

# dataset setup
training_data = grv.training_data

X = torch.Tensor(np.array([i[0] for i in training_data]))
Y = torch.Tensor(np.array([i[1] for i in training_data]))
P = torch.Tensor(np.array([i[2] for i in training_data]))
Y_m = softmin(torch.Tensor(np.array([i[3] for i in training_data])))
bfSP = torch.Tensor(np.array([i[4] for i in training_data]))

# Generate winner only class
Y_w = []
for i in Y:
    n = np.zeros(8)
    index = torch.argmin(i)
    n[index] = float(1)
    Y_w.append(n)

Y_w = torch.tensor([i for i in Y_w])
X = X.to(device)
Y_w = Y_w.to(device)


Y_m = Y_m.to(device)
P = P.to(device)
bfSP = bfSP.to('cuda')
bfSP = torch.nan_to_num(bfSP, nan=0)
my_dataset = TensorDataset(X, Y_m, P, bfSP)
my_dataloader = DataLoader(my_dataset)


C:\Users\Nick\AppData\Local\Temp\ipykernel_6712\2102313497.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:204.)
  Y_w = torch.tensor([i for i in Y_w])


In [6]:
# have to pass in dataset to get_data (created above)
def make_loader(dataset, config, train=True):
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(config["validation_split"] * dataset_size))
    random_seed = 42
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    if train:
        dataset_sampler = SubsetRandomSampler(train_indices)
        loader = torch.utils.data.DataLoader(
            dataset=dataset,
            batch_size=config["batch_size"],
            shuffle=False,
            pin_memory=False,
            num_workers=0,
            sampler=dataset_sampler
        )
    else:
        dataset_sampler = SubsetRandomSampler(val_indices)
        loader = torch.utils.data.DataLoader(
            dataset=dataset,
            shuffle=False,
            pin_memory=False,
            num_workers=0,
            sampler=dataset_sampler,
        )

    return loader


In [7]:
def build_network(f1_layer_size, f2_layer_size, dropout, num_layers=2):

    if num_layers == 2:
        network = nn.Sequential(  # fully-connected, dual hidden layer
            nn.Linear(120, f1_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f1_layer_size, f2_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f2_layer_size, 8),
            nn.Softmax(dim=1),
        )

    else:
        network = nn.Sequential(  # fully-connected, dual hidden layer
            nn.Linear(120, f1_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f1_layer_size, f2_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f2_layer_size, f2_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f2_layer_size, 8),
            nn.Softmax(dim=1),
        )


    return network


In [10]:
def custom_MSE(output, target):
    sorts = torch.argsort(target)
    out = sorts.narrow(1,0,3)
    ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)
    out_first3 = ohe*output
    target_ohe = ohe*target
    loss = torch.sum((((target_ohe+1)*abs(out_first3-target_ohe)+1)**2))
    # loss = torch.sum(((out_first3-target_ohe))**2)
    return loss
    


In [9]:
x = torch.tensor([1,2,3])

In [11]:
def custom_order_MSE(output, target):
    sorts = torch.argsort(target)
    out = sorts.narrow(1,0,3)
    ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)
    out_first3 = ohe*output
    target_ohe = ohe*target
    loss = torch.mean(((abs(out_first3-target_ohe)*10)**2))
    # loss = torch.sum(((out_first3-target_ohe))**2)
    return loss
    

In [24]:
def loss_customiser(x,y,loss_func=nn.HuberLoss(reduction='none',delta=0.1)):
    sorts_t = torch.argsort(y)
    out = sorts_t.narrow(1,0,3)
    ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)
    loss = loss_func(x,y)
    return torch.mean(loss)

In [26]:
def make(config, dataset):
    # Make the data

    train_loader = make_loader(dataset, config, train=True)
    test_loader = make_loader(dataset, config, train=False)
    # Make the model
    # model = Net().to(device)
    model = build_network(
        config["f1_layer_size"], config["f2_layer_size"], config["dropout"], config["num_layers"]
    )

    loss_functions = {
        "Huber":nn.HuberLoss(),
        "Huber_custom":loss_customiser,
        "MSE":nn.MSELoss(),
        "L1":nn.SmoothL1Loss(reduction='sum', beta=config["l1_beta"]),
        "BCE":nn.CrossEntropyLoss(),
        "Custom":custom_MSE,
        "KL":nn.KLDivLoss(reduction='batchmean'),
        "NLL":nn.NLLLoss()
    }
    # Make the loss and optimizer
    #  criterion = nn.NLLLoss()
    loss_f = loss_functions[config['loss']]
    criterion = loss_f
    optimizer = config["optimizer"]

    if optimizer == "sgd":
        optimizer = optim.SGD(
            model.parameters(), lr=config["learning_rate"], momentum=0.9
        )
    elif optimizer == "adam":
        optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])

    return model, train_loader, test_loader, criterion, optimizer


In [13]:
def test(model, test_loader, batch_ct):
    model.eval()
    classL, predL, maxL, correctL, priceP, priceR, bfPriceR, pred_odds, model_outputs = [], [], [], [], [], [], [], [], []
    # Run the model on some test examples
    with torch.no_grad():
        correct, total, max_sum, max_w_sum, profit, bfprofit, bfnotavail = 0, 0, 0, 0, 0,0,0
        value_pick_correct, value_pick_profit = 0, 0
        num_bets = 0
        for images, labels, prices, bfspPrices in test_loader:

            outputs = model(images)

            #converts prices from tensor to list
            prices = prices[0,].tolist()
            bfspPrices = bfspPrices[0,].tolist()

            #gets the prediction and its confidence and the real class
            max, predicted = torch.max(outputs.data, 1)
            _, real = torch.max(labels.data, 1)

            #converts prediction from tensor to item
            prediction = predicted.item()
            real_item = real.item()

            #appends prediction and likelyhood to lists
            predL.append(prediction)
            maxL.append(max.item())

        
            total += labels.size(0)
            correct += prediction == real_item

            correctL.append(int(prediction == real_item))
            classL.append(real_item)

            priceR.append(prices[real_item])
            priceP.append(prices[prediction])
            bfPriceR.append(bfspPrices[real_item])
            # print(outputs.data.flatten().tolist())

            predicted_odds = [
                1 / ((x + 10**-7)) for x in outputs.data.flatten().tolist()
            ]

            pred_odds.append(predicted_odds)
            model_outputs.append(outputs.data.flatten().tolist())

            if prices[real_item] > (predicted_odds[real_item] ):
                value_pick_correct += 1
                value_pick_profit += prices[real_item]

            bets = [x > (y) for x, y in zip(prices, predicted_odds)]
            num_bets += sum(bets)

            value_pick_profit += -sum(bets)

            if prediction == real_item:
                max_sum += max
                profit += prices[real_item]
                if bfspPrices[real_item]:
                    bfprofit += bfspPrices[real_item]
                else:
                    bfprofit += prices[real_item]
                    bfnotavail += 1
            else:
                max_w_sum += max

            profit += -1
            bfprofit += -1

            # print(f"{correct=}")

        # print(f"Accuracy of the model on the {total} " +
        #       f"test images: {100 * correct / total}%" +
        #       f"profit: {profit}"+
        #       f"profit: {value_pick_profit}")

        wandb.log(
            {
                "test_accuracy": correct / total,
                "correct_conf": max_sum / correct,
                "incorrect_conf": (max_w_sum) / (total - correct),
                "profit": profit,
                "bfprofit": bfprofit,
                "bfnotavail": bfnotavail,
                "value_pick_roi": value_pick_profit / num_bets,
                "num_bets_per": num_bets / total,
            }
        )

        # logdf = pd.DataFrame(
        #     data={
        #         "class": classL,
        #         "pred": predL,
        #         "max": maxL,
        #         "correct": correctL,
        #         "priceR": priceR,
        #         "priceP": priceP,
        #         "bets": sum(bets),
        #         "pred_odds": pred_odds,
        #         "model_outputs": model_outputs,
        #         "bfodds" : bfPriceR
        #     }
        # )
        # table = wandb.Table(dataframe=logdf)
        # wandb.log({"table_key": table})
        # classCounts = logdf["class"].value_counts()
        # predCounts = logdf["pred"].value_counts()
        # boxplot = logdf.boxplot(column=['priceR'],by='correct')
        # print(classCounts, predCounts)
        # boxplot
        # plt.savefig("boxplot.png")
        # wandb.log({"boxplot":boxplot})



In [14]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    #print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

In [15]:
def model_saver(model, optimizer, epoch, loss):
    
    pathtofolder = "C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model"
    model_name = wandb.run.name
    isExist = os.path.exists(
        f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
    )
    if isExist:
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": loss,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )
    else:
        print("created path")
        os.makedirs(
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
        )
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": loss,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )


In [16]:
def train(model, loader,test_loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0

    raw_inputs = True
    if config['loss'] == "KL":

        for epoch in tqdm(range(config.epochs)):
            for _, (images, labels, _ , _) in enumerate(loader):

                loss = train_batch_lsftmax(images, labels, model, optimizer, criterion, btch_count=batch_ct, raw_inputs=True)
                example_ct +=  len(images)
                batch_ct += 1

                # Report metrics every 25th batch
                if ((batch_ct + 1) % 250) == 0:
                    train_log(loss, example_ct, epoch)
                    

            if epoch %10 ==0:
                test(model,test_loader, epoch)
                model_saver(model,optimizer,epoch,loss)

    else:
        for epoch in tqdm(range(config.epochs)):
            for _, (images, labels, _ , _) in enumerate(loader):

                loss = train_batch(images, labels, model, optimizer, criterion, btch_count=batch_ct, raw_inputs=True)
                example_ct +=  len(images)
                batch_ct += 1

                # Report metrics every 25th batch
                if ((batch_ct + 1) % 250) == 0:
                    train_log(loss, example_ct, epoch)

            if epoch %10 ==0:
                test(model,test_loader, epoch)
                model_saver(model, optimizer, epoch, loss)

def train_batch(images, labels, model, optimizer, criterion, btch_count=0, raw_inputs=True):
    images, labels = images, labels
    

    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs, labels.float())
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss


def train_batch_lsftmax(images, labels, model, optimizer, criterion, btch_count=0, raw_inputs=True):
    images, labels = images, labels
    

    # Forward pass ➡
    outputs = model(images)
    loss = criterion(F.log_softmax(outputs), labels.float())
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [20]:
def model_pipeline(config=None,prev_model=None):
    dataset = my_dataset
    # tell wandb to get started
    with wandb.init(project="new customs", config=config):
      # access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)

      # make the model, data, and optimization problem
      model, train_loader, test_loader, criterion, optimizer = make(config, dataset)
      if prev_model:
        print("here")
        model.load_state_dict(torch.load(prev_model), strict = False)
      model = model.to(device)
      print(model)

      # and use them to train the model
      train(model, train_loader,test_loader, criterion, optimizer, config)

      # and test its final performance
      #test(model, test_loader)

    return model

In [18]:
sweep_config = {"method": "random"}

metric = {"name": "profit", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adam"},
    "f1_layer_size": {"values": [64]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3, 0.4, 0.5]},
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [500]},
        "validation_split": {"value": 0.1},
        "loss": {
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            'value': 'L1'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00005,
            "max": 0.0005,
        },
        "l1_beta": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0,
            "max": 1,
        },
        "batch_size": {
            'value': 32
            # "values": [32, 64, 128, 360,720]
            # 'values':[4,8,16,32,64,128,360]
        },
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config


{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'profit'},
 'parameters': {'batch_size': {'value': 32},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'values': [500]},
                'f1_layer_size': {'values': [64]},
                'f2_layer_size': {'values': [64]},
                'l1_beta': {'distribution': 'uniform', 'max': 1, 'min': 0},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.0005,
                                  'min': 5e-05},
                'loss': {'value': 'L1'},
                'num_layers': {'values': [2]},
                'optimizer': {'value': 'adam'},
                'validation_split': {'value': 0.1}}}


{'method': 'random',
 'metric': {'name': 'profit', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adam'},
  'f1_layer_size': {'values': [64]},
  'f2_layer_size': {'values': [64]},
  'dropout': {'values': [0.3, 0.4, 0.5]},
  'epochs': {'values': [500]},
  'validation_split': {'value': 0.1},
  'loss': {'value': 'L1'},
  'num_layers': {'values': [2]},
  'learning_rate': {'distribution': 'uniform', 'min': 5e-05, 'max': 0.0005},
  'l1_beta': {'distribution': 'uniform', 'min': 0, 'max': 1},
  'batch_size': {'value': 32}}}

In [19]:
sweep_config = {"method": "random"}

metric = {"name": "profit", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adam"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [256]},
    "dropout": {"values": [0.3, 0.4, 0.5]},
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [500]},
        "validation_split": {"value": 0.1},
        "loss": {
            # "values": [ "L1",  "Custom", "KL"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            'value': 'Custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00005,
            "max": 0.0005,
        },
        "l1_beta": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0,
            "max": 1,
        },
        "batch_size": {
            # 'value': 1000
            "values": [32, 64, 128, 360,720]
            # 'values':[4,8,16,32,64,128,360]
        },
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config


{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'profit'},
 'parameters': {'batch_size': {'values': [32, 64, 128, 360, 720]},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'values': [500]},
                'f1_layer_size': {'values': [256]},
                'f2_layer_size': {'values': [256]},
                'l1_beta': {'distribution': 'uniform', 'max': 1, 'min': 0},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.0005,
                                  'min': 5e-05},
                'loss': {'value': 'Custom'},
                'num_layers': {'values': [2]},
                'optimizer': {'value': 'adam'},
                'validation_split': {'value': 0.1}}}


{'method': 'random',
 'metric': {'name': 'profit', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adam'},
  'f1_layer_size': {'values': [256]},
  'f2_layer_size': {'values': [256]},
  'dropout': {'values': [0.3, 0.4, 0.5]},
  'epochs': {'values': [500]},
  'validation_split': {'value': 0.1},
  'loss': {'value': 'Custom'},
  'num_layers': {'values': [2]},
  'learning_rate': {'distribution': 'uniform', 'min': 5e-05, 'max': 0.0005},
  'l1_beta': {'distribution': 'uniform', 'min': 0, 'max': 1},
  'batch_size': {'values': [32, 64, 128, 360, 720]}}}

In [27]:
wandb_config = {'batch_size': 32, 'dropout': 0.5, 'epochs': 250, 'f1_layer_size': 128, 'f2_layer_size': 128, 'learning_rate': 0.0001, 'loss': 'Huber_custom', 'l1_beta':0.1, 'num_layers': 2, 'optimizer': 'adam', 'validation_split': 0.1}
model = model_pipeline(config=wandb_config)
# model = model_pipeline(config=wandb_config, prev_model=r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\good models\bumbling-sweep-6_430.pt')

  0%|          | 0/250 [00:00<?, ?it/s]

{'batch_size': 32, 'dropout': 0.5, 'epochs': 250, 'f1_layer_size': 128, 'f2_layer_size': 128, 'learning_rate': 0.0001, 'loss': 'Huber_custom', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adam', 'validation_split': 0.1}
250
{'batch_size': 32, 'dropout': 0.5, 'epochs': 250, 'f1_layer_size': 128, 'f2_layer_size': 128, 'learning_rate': 0.0001, 'loss': 'Huber_custom', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adam', 'validation_split': 0.1}
Sequential(
  (0): Linear(in_features=120, out_features=128, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=128, out_features=128, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=128, out_features=8, bias=True)
  (7): Softmax(dim=1)
)


  0%|          | 1/250 [00:16<1:09:10, 16.67s/it]

created path


100%|██████████| 250/250 [31:21<00:00,  7.53s/it]


bfnotavail,▁▇█████▇▇▇▇▇▇▇▇▇▇▆▆▇▆▆▆▆▆
bfprofit,▁▄▄▄▇▇▇▄█▆█▆▆▇▅▆▆▅▇█▄▇▇▄▄
correct_conf,▁▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇███▇▇████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
incorrect_conf,▁▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇▇████
loss,▇▆█▇▇▇▄▇▆▅▇▄▄▅▆▅▅▅▅▄▇▆▆▆▅▄▃▄▅█▃▂▃▄▆▁▅▃▇▆
num_bets_per,█▅▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
profit,▁▄▅▃▇▇▆▃▇▆█▆▆▆▅▆▆▅▆▇▃▆▆▄▄
test_accuracy,▁▇█████▇▇▇█▇▇▇▇▇▇▇▇▇▆▇▇▇▆
value_pick_roi,▁▆▅▄▄▃▄▆▅▃▆▄▆▅▆▅▆▆▇▅▅█▆▇▇
bfnotavail,866


In [26]:

#model_pipeline(config)
sweep_id = wandb.sweep(sweep_config, project="grv_priced_sweep_l1_test_new")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)


Create sweep with ID: icv88iyd
Sweep URL: https://wandb.ai/nickojelly/grv_priced_sweep_l1_test_new/sweeps/icv88iyd


wandb: Agent Starting Run: wisqv66j with config:
wandb: 	batch_size: 360
wandb: 	dropout: 0.4
wandb: 	epochs: 500
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 256
wandb: 	l1_beta: 0.5810159406207785
wandb: 	learning_rate: 0.00040775332522671894
wandb: 	loss: Custom
wandb: 	num_layers: 2
wandb: 	optimizer: adam
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


  0%|          | 0/500 [00:00<?, ?it/s]

{'batch_size': 360, 'dropout': 0.4, 'epochs': 500, 'f1_layer_size': 256, 'f2_layer_size': 256, 'l1_beta': 0.5810159406207785, 'learning_rate': 0.00040775332522671894, 'loss': 'Custom', 'num_layers': 2, 'optimizer': 'adam', 'validation_split': 0.1}
500
{'batch_size': 360, 'dropout': 0.4, 'epochs': 500, 'f1_layer_size': 256, 'f2_layer_size': 256, 'l1_beta': 0.5810159406207785, 'learning_rate': 0.00040775332522671894, 'loss': 'Custom', 'num_layers': 2, 'optimizer': 'adam', 'validation_split': 0.1}
Sequential(
  (0): Linear(in_features=120, out_features=256, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.4, inplace=False)
  (3): Linear(in_features=256, out_features=256, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.4, inplace=False)
  (6): Linear(in_features=256, out_features=8, bias=True)
  (7): Softmax(dim=1)
)


100%|██████████| 500/500 [24:04<00:00,  2.89s/it]


...

  0%|          | 0/500 [00:00<?, ?it/s]

{'batch_size': 64, 'dropout': 0.4, 'epochs': 500, 'f1_layer_size': 256, 'f2_layer_size': 256, 'l1_beta': 0.6804381740174369, 'learning_rate': 7.813183771033159e-05, 'loss': 'Custom', 'num_layers': 2, 'optimizer': 'adam', 'validation_split': 0.1}
500
{'batch_size': 64, 'dropout': 0.4, 'epochs': 500, 'f1_layer_size': 256, 'f2_layer_size': 256, 'l1_beta': 0.6804381740174369, 'learning_rate': 7.813183771033159e-05, 'loss': 'Custom', 'num_layers': 2, 'optimizer': 'adam', 'validation_split': 0.1}
Sequential(
  (0): Linear(in_features=120, out_features=256, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.4, inplace=False)
  (3): Linear(in_features=256, out_features=256, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.4, inplace=False)
  (6): Linear(in_features=256, out_features=8, bias=True)
  (7): Softmax(dim=1)
)


  0%|          | 1/500 [00:14<2:01:16, 14.58s/it]

created path


 45%|████▌     | 226/500 [19:19<21:15,  4.66s/it]

In [ ]:
model

Sequential(
  (0): Linear(in_features=120, out_features=256, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=256, out_features=32, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=32, out_features=8, bias=True)
  (7): Softmax(dim=1)
)